# Create training sub set
Create a random subset of training and val data either on a balanced set of chexpert diseases or on just one disease.

In [1]:
#hermione created: 14/07/2023
import pandas as pd
import numpy as np

train = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/train.csv")
chexpert = pd.read_csv("/home/hermione/Documents/data/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-chexpert.csv")
val = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/valid.csv")
train_chex = pd.merge(train, chexpert, on=['subject_id', 'study_id'])
val_chex = pd.merge(val, chexpert, on=['subject_id', 'study_id'])

summary = pd.DataFrame(columns = chexpert.columns[2:], index = ["chexpert", "train", "val"])
diseases = chexpert.columns[2:]
chex_dis = []
train_dis = []
val_dis = []

for col in diseases:
    #print(col)
    def dis_sum(data):
        return np.sum(data[data[col]==1][col].astype(int))
    summary[col] = [dis_sum(chexpert), dis_sum(train_chex), dis_sum(val_chex)]
    #print(np.sum(chexpert[chexpert[col]==1][col].astype(int)))
    #print(np.sum(train_chex[train_chex[col]==1][col].astype(int)))
    #print(np.sum(val_chex[val_chex[col]==1][col].astype(int)))

print(len(chexpert), len(train), len(val))
summary #potentially make percentage


227827 33510 4894


,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
chexpert,45808,44845,10778,27018,7179,4390,6284,51525,75455,54300,2011,16556,10358,66558
train,5110,4036,856,3003,613,638,801,6631,14514,5595,260,2150,1142,6161
val,753,611,104,499,70,75,111,984,2055,825,38,319,160,896


In [3]:
#select data
train_data_index = train_chex[train_chex["No Finding"] == 1].sample(frac=(1/3), random_state=1).index
val_data_index = val_chex[val_chex["No Finding"] == 1].sample(frac=(1/3), random_state=1).index
train_data = train_chex[train_chex.index.isin(train_data_index)| (train_chex["Pleural Effusion"] == 1)]

val_data = val_chex[val_chex.index.isin(val_data_index) | (val_chex["Pleural Effusion"] == 1)]

print(train_data.columns, len(train_data))
train_final = train_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
val_final = val_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
print(len(train_final), len(val_final))

Index(['subject_id', 'study_id', 'image_id', 'mimic_image_file_path',
       'reference_report', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
       'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax', 'Support Devices'],
      dtype='object') 10433
10433 1510


In [4]:
train_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/train_subset.csv",index=False)
val_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/valid_subset.csv", index=False)